In [1]:
# Instalar Selenium
!pip install selenium

### Paso 0: Importar librerías

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time

import pandas as pd

### Paso 1: Descargar y ubicar Chrome Driver
Se necesita descargar la última versión estable de ChromeDriver:
<br>
https://chromedriver.chromium.org/downloads
<br>

### Paso 2: Abrir y loguearte en WhatsApp Web

In [3]:
# Entrar a la página de WhatsApp Web
options = Options()
options.add_argument("--disable-extensions")
driver = webdriver.Chrome("chromedriver.exe", options=options) # Especificar la ruta del Chrome Driver (en este caso está ubicado en la misma ruta del código)
driver.get("https://web.whatsapp.com") 

# Para escanear el código QR tienes que entrar a 
# "Ajustes" -> Ícono de QR (a la derecha de tu nombre) -> "Escanear código" -> "Vincular dispositivo"
print("Escanea el código QR y luego presiona 'Enter'")
input()
print("Logueado!")

Escanea el código QR y luego presiona 'Enter'

Logueado!


### Paso 3: Leer el archivo que contiene los datos de las personas

In [4]:
# El archivo debe contener los números de teléfono y variables como su nombre, correo o lo que se considere pertinente en el mensaje
data = pd.read_excel('Datos.xlsx', sheet_name='Ventas')
data

# Note: Los números se cambiarán por prevención

,Persona,Sobrenombre,Fecha de compra,Correo,Producto,Importe,Moneda,Número
0,Juan Fernando Pérez del Corral,Juan,2021-03-21,juan.perez@gmail.com,Bitcoin,100,PEN,51966561101
1,Valentina Laverde de la Rosa,Vale,2020-12-02,valentina.laverde@outlook.com,Ethereum,200,EUR,51954616042
2,Óscar de la Renta,Óscar,2018-08-11,oscar.delarenta@yahoo.com,XRP,300,USD,51976304113
3,Sara Teresa Sánchez del Pinar,Sara,2020-11-09,sara.sanchez@hotmail.com,Terra,400,GBP,51940108810
4,Efraín de las Casas Mejía,Efraín,2019-07-23,efrain.delascasas@gmail.com,Polkadot,500,CHF,51991353065
5,Julieta Ponce de León,Julieta,2021-04-04,julieta.ponce@outlook.com,Polygon,600,JPY,51920221645
6,Martín Elías de los Ríos Acosta,Martín,2022-01-08,martin.delosrios@yahoo.com,Solana,700,HKD,51998102012
7,Gabriela de la Pava de la Torre,Gabriela,2019-04-17,gabriela.delapava@hotmail.com,Cardano,800,CAD,51971056831
8,Matías de Greiff Rincón,Matías,2022-01-01,matias.degreiff@gmail.com,DogeCoin,900,CNY,51957622897
9,Manuela del Pino Hincapié,Manuela,2021-12-04,manuela.delpino@outlook.com,Avalanche,1000,AUD,51977530679


### Paso 4, 5, 6, 7, 8 y 9: Enviar mensajes masivos y personalizados

In [5]:
for i in range(len(data)):
    
    # Paso 4: Ubicar la caja donde se buscan los nombres
    search = driver.find_element_by_xpath("//div[@role='textbox']")
    search.clear()
    
    # Paso 5: Introducir el número en el buscador
    contacto = data.loc[i, 'Número'].astype(str) # Convertir a string para que se añada al mensaje
    search.send_keys(contacto)
    time.sleep(2) # Esperar 2 segundos a que cargue
    
    # Paso 6: Seleccionar el contacto
    selected_contact = driver.find_element_by_xpath("//span[@class='_3q9s6']")
#     selected_contact = driver.find_elements_by_xpath("//div[@class='zoWT4']")[-1]
#     selected_contact = driver.find_elements_by_xpath("//span[@class='ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 l7jjieqr i0jNr']")[-2]
    selected_contact.click()
    
    # Paso 7: Formar el mensaje personalizado
    persona = data.loc[i,'Sobrenombre']
    importe = data.loc[i,'Importe'].astype(str) # Convertir a string para que se añada al mensaje
    moneda = data.loc[i,'Moneda']
    producto = data.loc[i,'Producto']
    fecha = data.loc[i,'Fecha de compra'].strftime("%m/%d/%Y") # Convertir a string para que se añada al mensaje
    correo = data.loc[i,'Correo']

    mensaje = (persona + "!, gracias por tu compra de " + importe + " " + moneda + " en " + producto + " el día " + fecha + " " + u'\u263A' + ". \n " +
              "Te enviaremos el comprobante de pago a tu correo " +  correo + "!") # Si quieres usar emoji, ten cuenta que
                                                                                   # Chrome Driver solo utiliza caracteres BMP
                                                                                   # Por ejemplo: 263A es ☺, 2764 es ❤, etc. 
                                                                                   # Encuentra más en https://codepoints.net/basic_multilingual_plane
        
    # Paso 8: Ubicar la caja en la que se digita el mensaje
    inp_xpath = '//div[@title="Escribe un mensaje aquí"]' 
    input_box = driver.find_element_by_xpath(inp_xpath)
    time.sleep(2) # Esperar 2 segundos a que cargue
    
    # Paso 9: Enviar el mensaje
    input_box.send_keys(mensaje + Keys.ENTER)
    time.sleep(2) # Esperar 2 segundos a que cargue

### (Opcional) Paso 10: Cerrar ventana de WhatsApp Web

In [6]:
# Cerrar ventana de WhatsApp Web al terminar
driver.quit()